In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/used-car-dataset-ford-and-mercedes/vw.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/bmw.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/skoda.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/unclean cclass.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/cclass.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/unclean focus.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/ford.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/focus.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/toyota.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/vauxhall.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/merc.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/audi.csv
/kaggle/input/used-car-dataset-ford-and-mercedes/hyundi.csv


## **Importing Libraries**

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

## **Reading desired dataset**

In [3]:
df = pd.read_csv("/kaggle/input/used-car-dataset-ford-and-mercedes/audi.csv")
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [ ]:
#Checking for null values 
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
#Checking counts of model columns
df['model'].value_counts()

In [4]:
#Changing year column to car age 
df["car_age"] = 2020 - df["year"]
df = df.drop(columns = ["year"])
df.head()

,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age
0,A1,12500,Manual,15735,Petrol,150,55.4,1.4,3
1,A6,16500,Automatic,36203,Diesel,20,64.2,2.0,4
2,A1,11000,Manual,29946,Petrol,30,55.4,1.4,4
3,A4,16800,Automatic,25952,Diesel,145,67.3,2.0,3
4,A3,17300,Manual,1998,Petrol,145,49.6,1.0,1


In [5]:
#Converting categorical variable into numerical form using label encoder.
#Other option includes One - hot Encoder 
le = LabelEncoder()
df["le_transmission"] = le.fit_transform(df["transmission"])
df["le_fuelType"] = le.fit_transform(df["fuelType"])
df.head(10)

,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age,le_transmission,le_fuelType
0,A1,12500,Manual,15735,Petrol,150,55.4,1.4,3,1,2
1,A6,16500,Automatic,36203,Diesel,20,64.2,2.0,4,0,0
2,A1,11000,Manual,29946,Petrol,30,55.4,1.4,4,1,2
3,A4,16800,Automatic,25952,Diesel,145,67.3,2.0,3,0,0
4,A3,17300,Manual,1998,Petrol,145,49.6,1.0,1,1,2
5,A1,13900,Automatic,32260,Petrol,30,58.9,1.4,4,0,2
6,A6,13250,Automatic,76788,Diesel,30,61.4,2.0,4,0,0
7,A4,11750,Manual,75185,Diesel,20,70.6,2.0,4,1,0
8,A3,10200,Manual,46112,Petrol,20,60.1,1.4,5,1,2
9,A1,12000,Manual,22451,Petrol,30,55.4,1.4,4,1,2


In [6]:
#Droping unnecessory columns 
df = df.drop(columns = ["transmission", "fuelType"])
df.head()

,model,price,mileage,tax,mpg,engineSize,car_age,le_transmission,le_fuelType
0,A1,12500,15735,150,55.4,1.4,3,1,2
1,A6,16500,36203,20,64.2,2.0,4,0,0
2,A1,11000,29946,30,55.4,1.4,4,1,2
3,A4,16800,25952,145,67.3,2.0,3,0,0
4,A3,17300,1998,145,49.6,1.0,1,1,2


In [7]:
# Convert categorical variable ("model") into dummy variable
df_encoded = pd.get_dummies(df,columns=['model'])
df_encoded.head()

,price,mileage,tax,mpg,engineSize,car_age,le_transmission,le_fuelType,model_ A1,model_ A2,...,model_ RS5,model_ RS6,model_ RS7,model_ S3,model_ S4,model_ S5,model_ S8,model_ SQ5,model_ SQ7,model_ TT
0,12500,15735,150,55.4,1.4,3,1,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1,16500,36203,20,64.2,2.0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11000,29946,30,55.4,1.4,4,1,2,1,0,...,0,0,0,0,0,0,0,0,0,0
3,16800,25952,145,67.3,2.0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17300,1998,145,49.6,1.0,1,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Splitting the data into training and testing
#It can be also split in three ways training, validation and testing sets during on your preference
X = df_encoded.drop(['price'],axis=1)
y = df_encoded['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## **Linear regression**

In [10]:
linreg = LinearRegression().fit(X_train, y_train)

print('linear model coeff (w): {}'
     .format(linreg.coef_))
print('linear model intercept (b): {:.3f}'
     .format(linreg.intercept_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))

linear model coeff (w): [-8.92256290e-02 -1.87728264e+01 -1.34351074e+02  5.70909460e+03
 -2.04788459e+03  5.83839414e+01  4.67488316e+02 -1.28987100e+04
  1.05884307e+04 -1.07490710e+04 -1.04576208e+04 -8.62092260e+03
 -7.80207468e+03 -7.06235441e+03 -3.16941465e+03 -1.05408617e+04
 -8.46670857e+03 -3.27577776e+03  4.20412255e+03  1.47062838e+04
  4.64331867e+04 -1.13233858e+03  9.73070012e+03  6.73567335e+03
  1.45270193e+04 -6.19820639e-10 -7.38480634e+03 -4.23270667e+02
 -6.59711895e+03 -5.87669780e+03 -1.40620220e+03  7.61911152e+03
 -8.68057735e+03]
linear model intercept (b): 36977.104
R-squared score (training): 0.873
R-squared score (test): 0.881


## **Ridge regression**

In [11]:
linridge = Ridge(alpha=20.0).fit(X_train, y_train)

print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

ridge regression linear model intercept: 29186.527463035643
ridge regression linear model coeff:
[-9.24204374e-02 -1.75030373e+01 -1.31847064e+02  7.86076933e+03
 -2.05656109e+03 -5.35906047e+01  7.15713389e+02 -8.32294029e+03
  7.49667217e+02 -6.70676416e+03 -7.09587412e+03 -5.43458707e+03
 -4.69735935e+03 -4.73394563e+03 -1.91017408e+03 -6.29198452e+03
 -4.77852838e+03 -4.11108822e+02  4.66412754e+03  1.15289090e+04
  2.19076341e+04  3.78831303e+02  5.81173601e+03  3.77611648e+03
  8.36247452e+03  0.00000000e+00 -1.98561674e+03  5.45547350e+02
 -2.79440155e+02 -1.25974653e+03  7.41055424e+01  1.32070637e+03
 -5.21178565e+03]
R-squared score (training): 0.858
R-squared score (test): 0.865
Number of non-zero features: 32


In [ ]:
linridge = Ridge(alpha=20.0).fit(X_train_scaled, y_train)

print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

In [12]:
print('Ridge regression: effect of alpha regularization parameter\n')
for this_alpha in [1, 10, 20, 50]:
    linridge = Ridge(alpha = this_alpha).fit(X_train_scaled, y_train)
    r2_train = linridge.score(X_train_scaled, y_train)
    r2_test = linridge.score(X_test_scaled, y_test)
    num_coeff_bigger = np.sum(abs(linridge.coef_) > 1.0)
    print('Alpha = {:.2f}\nnum abs(coeff) > 1.0: {}, \
r-squared training: {:.2f}, r-squared test: {:.2f}\n'
         .format(this_alpha, num_coeff_bigger, r2_train, r2_test))

Ridge regression: effect of alpha regularization parameter

Alpha = 1.00
num abs(coeff) > 1.0: 32, r-squared training: 0.87, r-squared test: 0.88

Alpha = 10.00
num abs(coeff) > 1.0: 32, r-squared training: 0.87, r-squared test: 0.88

Alpha = 20.00
num abs(coeff) > 1.0: 32, r-squared training: 0.87, r-squared test: 0.88

Alpha = 50.00
num abs(coeff) > 1.0: 32, r-squared training: 0.87, r-squared test: 0.88



## **Lasso Regression**



In [13]:
linlasso = Lasso(alpha=2.0, max_iter = 10000).fit(X_train_scaled, y_train)

print('lasso regression linear model intercept: {}'
     .format(linlasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(linlasso.coef_))
print('Non-zero features: {}'
     .format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'
     .format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'
     .format(linlasso.score(X_test_scaled, y_test)))

lasso regression linear model intercept: 22822.755331614717
lasso regression linear model coeff:
[-2105.36725511 -1263.1007829  -1759.37464058  3434.24279885
 -4428.81950073    42.17153018   461.21506234 -1469.21297195
   204.35837314  -874.10685563  -663.29337098   -40.54363898
   166.60441572   148.41751812   546.24967542  -546.09698107
     0.          1437.53522347  2361.5696596   1834.54349449
  2716.03866136   417.01715636  1037.71241729   768.32768945
  1486.40531443     0.            44.85330507   298.81902662
    17.88798842    53.2388464    250.57097022   386.4789401
   -33.84842723]
Non-zero features: 31
R-squared score (training): 0.873
R-squared score (test): 0.881



## **Support Vector Regression[](http://)**

In [21]:
from sklearn.svm import SVR
pipeline_svr=Pipeline([('scalar1',StandardScaler()),
                     ('pca1',PCA(n_components=2)),
                     ('lr_classifier',SVR(kernel='linear'))])
pipeline_svr.fit(X_train, y_train)
pipeline_svr.score(X_test,y_test)

0.5570504474828746

## **Decision Tree Regression**

In [14]:
dt = DecisionTreeRegressor(random_state=0)

#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))


#fit model
dt = dt.fit(X_train,y_train.values.ravel())
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

#print score
print('dt train score %.3f, dt test score: %.3f' % (
dt.score(X_train,y_train),
dt.score(X_test, y_test)))

dt train score 0.999, dt test score: 0.910


## **Random Forest Regression**


In [15]:
forest = RandomForestRegressor(n_estimators = 100,
                              criterion = 'mse',
                              random_state = 1,
                              n_jobs = -1)
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))

#fit model
forest.fit(X_train,y_train.values.ravel())
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

#print score
print('forest train score %.3f, forest test score: %.3f' % (
forest.score(X_train,y_train),
forest.score(X_test, y_test)))

forest train score 0.993, forest test score: 0.956


## **Gradient Boosting Regression**

In [16]:
GB=GradientBoostingRegressor(random_state=0)
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))


#fit model
gb = GB.fit(X_train,y_train.values.ravel())
y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)

#print score
print('GB train score %.3f, GB test score: %.3f' % (
gb.score(X_train,y_train),
gb.score(X_test, y_test)))

GB train score 0.950, GB test score: 0.942


## **XGB Regression** 

In [17]:
XGB=XGBRegressor(random_state=0)
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))


#fit model
xgb = XGB.fit(X_train,y_train.values.ravel())
y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)

#print score
print('XGB train score %.3f, XGB test score: %.3f' % (
xgb.score(X_train,y_train),
xgb.score(X_test, y_test)))

XGB train score 0.985, XGB test score: 0.953


## **AdaBoost Regression**

In [19]:
ABR=AdaBoostRegressor(random_state=0)
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))


#fit model
abr = ABR.fit(X_train,y_train.values.ravel())
y_train_pred = abr.predict(X_train)
y_test_pred = abr.predict(X_test)

#print score
print('ABR train score %.3f, ABR test score: %.3f' % (
abr.score(X_train,y_train),
abr.score(X_test,y_test)))

ABR train score 0.765, ABR test score: 0.726


**We notice during the comparion of various regressions score
* Random Forest Regression : 95.6
* **eXtreme Gradient Boost Regression : 95.3

Hyperparameter tuning can be performed on both of these regressor.
**

**Hyperparameter Tuning**

In [23]:
def print_best_params(gd_model):
    param_dict = gd_model.best_estimator_.get_params()
    model_str = str(gd_model.estimator).split('(')[0]
    print("\n*** {} Best Parameters ***".format(model_str))
    for k in param_dict:
        print("{}: {}".format(k, param_dict[k]))
    print()

In [35]:
param_grid_rf = dict(n_estimators=[20],
                     max_depth=np.arange(1, 15, 2),
                     min_samples_split=[2],
                     min_samples_leaf= np.arange(1, 15, 2, int),
                     bootstrap=[True, False],
                     oob_score=[False, ])


forest = GridSearchCV(RandomForestRegressor(random_state=0), param_grid=param_grid_rf, cv=10, verbose=3)

#fit model
forest.fit(X_train,y_train.values.ravel())


#print score
print('\n\nforest train score %.3f, forest test score: %.3f' % (
forest.score(X_train,y_train),
forest.score(X_test, y_test)))

print_best_params(forest)

Fitting 10 folds for each of 98 candidates, totalling 980 fits
[CV] bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.345, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.402, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.382, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.361, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.387, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=

[CV]  bootstrap=True, max_depth=1, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False, score=0.416, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=1, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False, score=0.443, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=1, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False, score=0.397, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=1, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False, score=0.345, total=   0.1s
[CV] bootstrap=True, max_depth=1, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=

[CV]  bootstrap=True, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.687, total=   0.1s
[CV] bootstrap=True, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.724, total=   0.1s
[CV] bootstrap=True, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.739, total=   0.1s
[CV] bootstrap=True, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=True, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.689, total=   0.1s
[CV] bootstrap=True, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=

...

[CV]  bootstrap=False, max_depth=11, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False, score=0.928, total=   0.5s
[CV] bootstrap=False, max_depth=11, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=11, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False, score=0.760, total=   0.4s
[CV] bootstrap=False, max_depth=11, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=11, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False, score=0.917, total=   0.4s
[CV] bootstrap=False, max_depth=11, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=11, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False, score=0.939, total=   0.4s
[CV] bootstrap=False, max_depth=11, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False 


[CV]  bootstrap=False, max_depth=13, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.955, total=   0.5s
[CV] bootstrap=False, max_depth=13, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=13, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.929, total=   0.5s
[CV] bootstrap=False, max_depth=13, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=13, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.954, total=   0.5s
[CV] bootstrap=False, max_depth=13, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=13, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False, score=0.930, total=   0.5s
[CV] bootstrap=False, max_depth=13, min_samples_leaf=1, min_samples_split=2, n_estimators=20, oob_score=False 


[CV]  bootstrap=False, max_depth=13, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False, score=0.916, total=   0.4s
[CV] bootstrap=False, max_depth=13, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=13, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False, score=0.950, total=   0.4s
[CV] bootstrap=False, max_depth=13, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=13, min_samples_leaf=7, min_samples_split=2, n_estimators=20, oob_score=False, score=0.934, total=   0.5s
[CV] bootstrap=False, max_depth=13, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False 
[CV]  bootstrap=False, max_depth=13, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False, score=0.930, total=   0.4s
[CV] bootstrap=False, max_depth=13, min_samples_leaf=9, min_samples_split=2, n_estimators=20, oob_score=False 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 980 out of 980 | elapsed:  4.3min finished




forest train score 0.983, forest test score: 0.955

*** RandomForestRegressor Best Parameters ***
bootstrap: True
ccp_alpha: 0.0
criterion: mse
max_depth: 13
max_features: auto
max_leaf_nodes: None
max_samples: None
min_impurity_decrease: 0.0
min_impurity_split: None
min_samples_leaf: 1
min_samples_split: 2
min_weight_fraction_leaf: 0.0
n_estimators: 20
n_jobs: None
oob_score: False
random_state: 0
verbose: 0
warm_start: False



In [36]:
xgb1 = XGBRegressor()
parameters = {'nthread':[2], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [200]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 10,
                        n_jobs = -1,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   57.8s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.9min finished


[07:18:15] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[07:18:15] WARNING: /workspace/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[07:18:17] WARNING: /workspace/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
0.9617717084396699
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 200, 'nthread': 2, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


## **XG Boost Regression**
96.17717084396699
* Parameters: 
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 200, 'nthread': 2, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}